In [44]:
import pandas as pd

# 파일 경로
file_path = "../../data/통합_train_데이터.parquet"
df = pd.read_parquet(file_path)

In [45]:
top_ab = ['할부금액_3M_R12M',
 '이용금액_할부_무이자_R12M',
 '이용건수_할부_무이자_R12M',
 '정상입금원금_B0M',
 '이용금액_오프라인_R6M',
 '정상청구원금_B0M',
 '이용금액_오프라인_R3M',
 '_1순위카드이용금액',
 '평잔_일시불_해외_6M',
 '승인거절건수_입력오류_R3M',
 '청구금액_R3M',
 '이용금액_일시불_R12M',
 '이용금액_할부_무이자_R3M',
 '이용금액_할부_무이자_R6M',
 '정상입금원금_B5M',
 '이용금액_할부_R12M',
 '마일_적립포인트_R3M',
 '정상청구원금_B2M',
 '포인트_마일리지_환산_B0M',
 '청구금액_B0',
 '정상입금원금_B2M',
 '할부건수_무이자_3M_R12M',
 '정상청구원금_B5M',
 '청구금액_R6M',
 '여유_숙박이용금액',
 '최대이용금액_일시불_R12M',
 '_1순위업종_이용금액',
 '잔액_할부_B0M',
 '할부금액_무이자_3M_R12M',
 '잔액_할부_무이자_B0M']

In [46]:
top_cde = ['이용금액_일시불_R3M',
 '이용금액_R3M_신용체크',
 '정상입금원금_B0M',
 '이용금액_오프라인_R6M',
 '정상청구원금_B0M',
 '이용건수_신용_R6M',
 '이용금액_오프라인_R3M',
 '이용건수_일시불_R12M',
 '_1순위카드이용금액',
 '이용금액_오프라인_B0M',
 '청구금액_R3M',
 '이용금액_일시불_R12M',
 '이용금액_R3M_신용',
 '정상입금원금_B5M',
 '이용건수_오프라인_B0M',
 '정상청구원금_B2M',
 '이용건수_신판_R12M',
 '청구금액_B0',
 '정상입금원금_B2M',
 '이용금액_일시불_B0M',
 '정상청구원금_B5M',
 '청구금액_R6M',
 '최대이용금액_일시불_R12M',
 '이용가맹점수',
 '이용건수_신용_R12M',
 '이용금액_일시불_R6M']

In [47]:
# 📌 PC1~PC5에서 반복적으로 중요한 변수:
pca_cols = ['CA이자율_할인전', 'CL이자율_할인전', 'RV_평균잔액_R3M', 'RV일시불이자율_할인전', 'RV최소결제비율', 'RV현금서비스이자율_할인전', '방문월수_앱_R6M', '방문일수_앱_B0M', '방문일수_앱_R6M', '방문횟수_앱_B0M', '방문후경과월_앱_R6M', '이용금액_R3M_신용', '이용금액_R3M_신용체크', '이용금액_일시불_B0M', '이용금액대', '일시불ONLY전환가능여부', '잔액_리볼빙일시불이월_B0M', '잔액_일시불_B0M', '잔액_일시불_B1M', '잔액_일시불_B2M', '잔액_카드론_B0M', '잔액_카드론_B1M', '잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M', '잔액_카드론_B5M', '정상청구원금_B0M', '정상청구원금_B2M', '정상청구원금_B5M', '청구금액_B0', '청구금액_R3M', '청구금액_R6M', '최종카드론_대출금액', '카드론이용금액_누적', '평잔_RV일시불_3M', '평잔_RV일시불_6M', '평잔_일시불_3M', '평잔_일시불_6M', '평잔_카드론_3M', '평잔_카드론_6M', '평잔_할부_3M', '홈페이지_금융건수_R3M', '홈페이지_금융건수_R6M', '홈페이지_선결제건수_R3M', '홈페이지_선결제건수_R6M']

In [48]:
selected_cols=(top_ab + top_cde + pca_cols)
selected_cols = list(dict.fromkeys(selected_cols))

In [49]:
def map_categorical_columns(df, verbose=True):
    """
    미리 정의된 매핑 기준에 따라 범주형 컬럼들을 수치형으로 변환합니다.
    처리 컬럼: 거주시도명, 연회비발생카드수_B0M, 한도증액횟수_R12M, 이용금액대,
              할인건수_R3M, 할인건수_B0M, 방문횟수_PC_R6M, 방문횟수_앱_R6M, 방문일수_PC_R6M
    """

    # 1. 거주시도명 → 수도권 여부
    capital_area = ['서울', '경기', '인천']
    if '거주시도명' in df.columns:
        df['거주시도명'] = df['거주시도명'].apply(lambda x: 1 if x in capital_area else 0)

    # 2. 연회비발생카드수_B0M
    mapping = {"0개": 0, "1개이상": 1}
    if '연회비발생카드수_B0M' in df.columns:
        df['연회비발생카드수_B0M'] = df['연회비발생카드수_B0M'].map(mapping).astype(int)
        if verbose: print("[연회비발생카드수_B0M] 인코딩 완료")

    # 3. 한도증액횟수_R12M
    mapping = {"0회": 0, "1회이상": 1}
    if '한도증액횟수_R12M' in df.columns:
        df['한도증액횟수_R12M'] = df['한도증액횟수_R12M'].map(mapping).astype(int)
        if verbose: print("[한도증액횟수_R12M] 인코딩 완료")

    # 4. 이용금액대 (중간값 기준: 만원 단위)
    mapping = {
        "09.미사용": 0,
        "05.10만원-": 5,
        "04.10만원+": 20,
        "03.30만원+": 40,
        "02.50만원+": 75,
        "01.100만원+": 150
    }
    if '이용금액대' in df.columns:
        df['이용금액대'] = df['이용금액대'].map(mapping)
        if verbose: print("[이용금액대] 중간값 인코딩 완료")

   # 5. 할인건수 인코딩
    discount_map = {
        "1회 이상": 1,
        "10회 이상": 10,
        "20회 이상": 20,
        "30회 이상": 30,
        "40회 이상": 40
    }
    for col in ['할인건수_R3M', '할인건수_B0M']:
        if col in df.columns:
            df[col] = df[col].map(discount_map).astype(int)
            if verbose: print(f"[{col}] 인코딩 완료")

    # 6. 방문횟수 및 방문일수 인코딩
    visit_map = {
        "1회 이상": 1,
        "10회 이상": 10,
        "20회 이상": 20,
        "30회 이상": 30,
        "40회 이상": 40,
        "50회 이상": 50,
        "60회 이상": 60,
        "70회 이상": 70,
        "80회 이상": 80
    }

    visit_cols = ['방문횟수_PC_R6M', '방문횟수_앱_R6M', '방문일수_PC_R6M']
    for col in visit_cols:
        if col in df.columns:
            df[col] = df[col].map(visit_map).astype(int)
            if verbose: print(f"[{col}] 인코딩 완료")

    return df

In [50]:
df = map_categorical_columns(df)

[연회비발생카드수_B0M] 인코딩 완료
[한도증액횟수_R12M] 인코딩 완료
[이용금액대] 중간값 인코딩 완료
[할인건수_R3M] 인코딩 완료
[할인건수_B0M] 인코딩 완료
[방문횟수_PC_R6M] 인코딩 완료
[방문횟수_앱_R6M] 인코딩 완료
[방문일수_PC_R6M] 인코딩 완료


In [51]:
ab_cols=(top_ab + pca_cols)
ab_cols = list(dict.fromkeys(ab_cols))

In [52]:
# 📌 Segment 라벨 기준: A/B → 1, C/D/E → 0
df['is_ab'] = df['Segment'].map(lambda x: 1 if x in ['A', 'B'] else 0)

# 👇 예시용 피처 컬럼 리스트 (사용 중인 걸로 대체하세요)
X = df[selected_cols]  # 선택된 피처들
y = df['is_ab']       # 이진 타겟

# ✅ Train / Validation 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ✅ 모델 후보 정의
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, confusion_matrix

models = {
#    "Logistic": LogisticRegression(max_iter=1000, class_weight='balanced'),
#    "RandomForest": RandomForestClassifier(n_estimators=100, class_weight='balanced'),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=(y == 0).sum() / (y == 1).sum()),
#    "LightGBM": LGBMClassifier(class_weight='balanced')
}

# ✅ 모델 학습 및 성능 평가
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds)
    print(f"\n📌 [{name}] F1-score: {f1:.4f}")
    print(classification_report(y_val, preds))
    print(confusion_matrix(y_val, preds))


c:\Users\somee\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:09:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📌 [XGBoost] F1-score: 0.5591
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    479777
           1       0.43      0.82      0.56       223

    accuracy                           1.00    480000
   macro avg       0.71      0.91      0.78    480000
weighted avg       1.00      1.00      1.00    480000

[[479531    246]
 [    41    182]]


In [53]:
# A/B 추려진 데이터에서
ab_df = df[df['is_ab'] == 1].copy()

# 타겟: A=0, B=1
y_ab = ab_df['Segment'].map({'A': 0, 'B': 1})

# 전체 피처셋
X_ab_all = ab_df[ab_cols]

In [54]:
from xgboost import XGBClassifier
import pandas as pd

# 간단한 XGBoost 모델 학습
xgb_ab = XGBClassifier()
xgb_ab.fit(X_ab_all, y_ab)

# 중요도 추출
importances = pd.Series(xgb_ab.feature_importances_, index=X_ab_all.columns)
top_features_ab = importances.sort_values(ascending=False).head(20).index.tolist()

print("📌 A/B에서 가장 중요한 피처 Top 20:")
print(top_features_ab)

📌 A/B에서 가장 중요한 피처 Top 20:
['정상입금원금_B0M', '이용금액_할부_무이자_R12M', '홈페이지_선결제건수_R3M', '평잔_일시불_6M', '정상입금원금_B2M', '할부건수_무이자_3M_R12M', '이용금액_R3M_신용', 'RV현금서비스이자율_할인전', '잔액_리볼빙일시불이월_B0M', '잔액_카드론_B0M', '평잔_RV일시불_6M', '이용금액_일시불_R12M', '포인트_마일리지_환산_B0M', '잔액_할부_B0M', 'CA이자율_할인전', '할부금액_무이자_3M_R12M', '방문월수_앱_R6M', '평잔_할부_3M', '이용건수_할부_무이자_R12M', 'CL이자율_할인전']


In [55]:
# 선택된 피처만 사용
X_ab = X_ab_all[top_features_ab]

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train_ab, X_val_ab, y_train_ab, y_val_ab = train_test_split(X_ab, y_ab, stratify=y_ab, test_size=0.2, random_state=42)

model_ab_final = XGBClassifier()
model_ab_final.fit(X_train_ab, y_train_ab)
preds_ab = model_ab_final.predict(X_val_ab)

print(classification_report(y_val_ab, preds_ab))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97       195
           1       0.86      0.66      0.75        29

    accuracy                           0.94       224
   macro avg       0.91      0.82      0.86       224
weighted avg       0.94      0.94      0.94       224



In [57]:
# ab_df를 X_ab와 같은 인덱스 기준으로 정렬
ab_df = ab_df.loc[X_ab.index].copy()

# 예측값 저장
ab_preds = model_ab_final.predict(X_ab)
ab_df['Segment_pred'] = ab_preds
ab_df['Segment_pred'] = ab_df['Segment_pred'].map({0: 'A', 1: 'B'})


In [58]:
cde_cols=(top_cde + pca_cols)
cde_cols = list(dict.fromkeys(cde_cols))

In [59]:
cols_needed = cde_cols + ['Segment', 'ID']
cde_df = df.loc[df['is_ab'] == 0, cols_needed]


In [60]:
# 타겟: Segment 문자 → 숫자 라벨
y_cde = cde_df['Segment'].map({'C': 0, 'D': 1, 'E': 2})

# 입력 피처
X_cde = cde_df[cde_cols]

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

# 학습용 데이터 분리
X_train_cde, X_val_cde, y_train_cde, y_val_cde = train_test_split(
    X_cde, y_cde, stratify=y_cde, test_size=0.2, random_state=42
)

# 다중 분류 모델 학습
model_cde = XGBClassifier(objective='multi:softmax', num_class=3, eval_metric='mlogloss')
model_cde.fit(X_train_cde, y_train_cde)

# 예측 및 평가
preds_cde = model_cde.predict(X_val_cde)
print(classification_report(y_val_cde, preds_cde))

              precision    recall  f1-score   support

           0       0.72      0.58      0.65     25518
           1       0.69      0.62      0.65     69848
           2       0.94      0.97      0.95    384411

    accuracy                           0.89    479777
   macro avg       0.78      0.72      0.75    479777
weighted avg       0.89      0.89      0.89    479777



In [62]:
# 전체 예측 (학습용 데이터 전체로)
cde_preds = model_cde.predict(X_cde)

# 숫자 → 문자 라벨 복원
cde_df['Segment_pred'] = pd.Series(cde_preds, index=X_cde.index)
cde_df['Segment_pred'] = cde_df['Segment_pred'].map({0: 'C', 1: 'D', 2: 'E'})

In [63]:
# ab_df는 이전에 Segment_pred가 포함된 A/B 예측 결과
final_df = pd.concat([ab_df, cde_df])

# ID 기준 정렬
final_df = final_df.sort_values('ID')

# 제출 파일 생성
submission = final_df[['ID', 'Segment_pred']].rename(columns={'Segment_pred': 'Segment'})
submission.to_csv('final_submission.csv', index=False)

In [83]:
test_df = pd.read_parquet("../../data/통합_test_데이터.parquet")
X_test = test_df[selected_cols].copy()  # 또는 ab_cols, cde_cols 기반

# ID 백업
test_ids = test_df['ID']

In [84]:
test_df[selected_cols]

,할부금액_3M_R12M,이용금액_할부_무이자_R12M,이용건수_할부_무이자_R12M,정상입금원금_B0M,이용금액_오프라인_R6M,정상청구원금_B0M,이용금액_오프라인_R3M,_1순위카드이용금액,평잔_일시불_해외_6M,승인거절건수_입력오류_R3M,...,평잔_RV일시불_6M,평잔_일시불_3M,평잔_일시불_6M,평잔_카드론_3M,평잔_카드론_6M,평잔_할부_3M,홈페이지_금융건수_R3M,홈페이지_금융건수_R6M,홈페이지_선결제건수_R3M,홈페이지_선결제건수_R6M
0,0,0,0,3680,24747,6495,19611,13852,384,0,...,0,3841,2933,0,0,0,0,0,0,0
1,10005,17396,23,8726,7896,9644,5489,11065,0,0,...,0,844,657,25782,31707,6514,0,0,0,0
2,21312,20756,9,11297,90530,11519,41402,27071,0,0,...,0,5606,6253,0,0,11865,0,0,0,0
3,5912,5136,0,1375,4683,1375,5343,4827,0,0,...,0,1510,819,0,0,704,0,0,0,0
4,13983,9884,8,3951,9920,3951,7567,8011,0,0,...,0,1373,979,0,0,3561,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599996,0,0,0,205,2331,205,1178,1231,0,0,...,0,237,137,0,0,0,3,3,3,7
599997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599998,1994,12509,4,0,89973,26308,44309,63592,240,0,...,0,17039,18108,0,0,1623,4,7,12,22


In [85]:
test_df = map_categorical_columns(test_df)

[연회비발생카드수_B0M] 인코딩 완료
[한도증액횟수_R12M] 인코딩 완료
[이용금액대] 중간값 인코딩 완료
[할인건수_R3M] 인코딩 완료
[할인건수_B0M] 인코딩 완료
[방문횟수_PC_R6M] 인코딩 완료
[방문횟수_앱_R6M] 인코딩 완료
[방문일수_PC_R6M] 인코딩 완료


In [91]:
# ✅ [STEP 0] 준비된 모델: model_ab_final (AB vs CDE), model_ab_final (A/B), model_cde (C/D/E)
# ✅ [STEP 1] test_df 전처리 완료되어 있다고 가정

# ✅ [STEP 2] A/B vs C/D/E 이진 분류 예측 (model_ab_final)
X_test_ab_vs_cde = test_df[top_ab + pca_cols]  # A/B vs C/D/E 모델에 사용된 피처만
X_test_ab_vs_cde = X_test_ab_vs_cde.loc[:, ~X_test_ab_vs_cde.columns.duplicated()]

ab_proba_test = model_ab_final.predict_proba(X_test_ab_vs_cde)[:, 1]
ab_pred_test = (ab_proba_test >= 0.5).astype(int)

# ✅ [STEP 3] A/B or C/D/E 분기
ab_idx = X_test_ab_vs_cde.index[ab_pred_test == 1]   # A/B로 예측된 인덱스
cde_idx = X_test_ab_vs_cde.index[ab_pred_test == 0]  # C/D/E로 예측된 인덱스

# ✅ [STEP 4] A/B 예측 (model_ab_final)
X_ab_test = test_df.loc[ab_idx, top_ab + pca_cols]
X_ab_test = X_ab_test.loc[:, ~X_ab_test.columns.duplicated()]

ab_result = model_ab_final.predict(X_ab_test)
ab_result = pd.Series(ab_result, index=ab_idx).map({0: 'A', 1: 'B'})

# ✅ [STEP 5] C/D/E 예측 (model_cde)
X_cde_test = test_df.loc[cde_idx, top_cde + pca_cols]
X_cde_test = X_cde_test.loc[:, ~X_cde_test.columns.duplicated()]

cde_result = model_cde.predict(X_cde_test)
cde_result = pd.Series(cde_result, index=cde_idx).map({0: 'C', 1: 'D', 2: 'E'})

# ✅ [STEP 6] 결과 통합 및 제출파일 생성
segment_pred = pd.concat([ab_result, cde_result]).sort_index()
submission = pd.DataFrame({'ID': test_df['ID'], 'Segment': segment_pred})
submission.to_csv('final_submission.csv', index=False)


ValueError: feature_names mismatch: ['정상입금원금_B0M', '이용금액_할부_무이자_R12M', '홈페이지_선결제건수_R3M', '평잔_일시불_6M', '정상입금원금_B2M', '할부건수_무이자_3M_R12M', '이용금액_R3M_신용', 'RV현금서비스이자율_할인전', '잔액_리볼빙일시불이월_B0M', '잔액_카드론_B0M', '평잔_RV일시불_6M', '이용금액_일시불_R12M', '포인트_마일리지_환산_B0M', '잔액_할부_B0M', 'CA이자율_할인전', '할부금액_무이자_3M_R12M', '방문월수_앱_R6M', '평잔_할부_3M', '이용건수_할부_무이자_R12M', 'CL이자율_할인전'] ['할부금액_3M_R12M', '이용금액_할부_무이자_R12M', '이용건수_할부_무이자_R12M', '정상입금원금_B0M', '이용금액_오프라인_R6M', '정상청구원금_B0M', '이용금액_오프라인_R3M', '_1순위카드이용금액', '평잔_일시불_해외_6M', '승인거절건수_입력오류_R3M', '청구금액_R3M', '이용금액_일시불_R12M', '이용금액_할부_무이자_R3M', '이용금액_할부_무이자_R6M', '정상입금원금_B5M', '이용금액_할부_R12M', '마일_적립포인트_R3M', '정상청구원금_B2M', '포인트_마일리지_환산_B0M', '청구금액_B0', '정상입금원금_B2M', '할부건수_무이자_3M_R12M', '정상청구원금_B5M', '청구금액_R6M', '여유_숙박이용금액', '최대이용금액_일시불_R12M', '_1순위업종_이용금액', '잔액_할부_B0M', '할부금액_무이자_3M_R12M', '잔액_할부_무이자_B0M', 'CA이자율_할인전', 'CL이자율_할인전', 'RV_평균잔액_R3M', 'RV일시불이자율_할인전', 'RV최소결제비율', 'RV현금서비스이자율_할인전', '방문월수_앱_R6M', '방문일수_앱_B0M', '방문일수_앱_R6M', '방문횟수_앱_B0M', '방문후경과월_앱_R6M', '

In [94]:
# 진짜 train 데이터가 저장된 변수로 다시 지정
train_df = df.copy()

# 그 다음 C/D/E 필터링
cde_df = train_df[train_df['Segment'].isin(['C', 'D', 'E'])].copy()

MemoryError: Unable to allocate 13.4 GiB for an array with shape (747, 2400000) and data type int64

In [93]:
# ✅ A/B vs C/D/E 이진 분류 학습
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# 1. A/B vs CDE 타겟 생성
y_ab_vs_cde = df['Segment'].map(lambda x: 1 if x in ['A', 'B'] else 0)
X_ab_vs_cde = df[selected_cols]  # 🔄 변경: selected_cols 사용
X_ab_vs_cde = X_ab_vs_cde.loc[:, ~X_ab_vs_cde.columns.duplicated()]

# 2. 학습/검증 분리
tX, vX, ty, vy = train_test_split(X_ab_vs_cde, y_ab_vs_cde, test_size=0.2, random_state=42, stratify=y_ab_vs_cde)

# 3. 모델 학습
model_ab_vs_cde = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
model_ab_vs_cde.fit(tX, ty)


# ✅ A vs B 이진 분류 학습 (A/B에 해당하는 샘플만 사용)
ab_df = df[df['Segment'].isin(['A', 'B'])].copy()
X_ab = ab_df[top_ab + pca_cols]  # 🔄 변경: top_ab + pca_cols 사용
X_ab = X_ab.loc[:, ~X_ab.columns.duplicated()]
y_ab = ab_df['Segment'].map({'A': 0, 'B': 1})

model_ab_final = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
model_ab_final.fit(X_ab, y_ab)


# ✅ C vs D vs E 다중 분류 학습
cde_df = df[df['Segment'].isin(['C', 'D', 'E'])].copy()
X_cde = cde_df[top_cde + pca_cols]  # 🔄 변경: top_cde + pca_cols 사용
X_cde = X_cde.loc[:, ~X_cde.columns.duplicated()]
y_cde = cde_df['Segment'].map({'C': 0, 'D': 1, 'E': 2})

model_cde = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
model_cde.fit(X_cde, y_cde)


c:\Users\somee\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:35:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\somee\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:36:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


MemoryError: Unable to allocate 13.4 GiB for an array with shape (750, 2398884) and data type int64